In [1]:
import os
 

In [2]:
%pwd

'c:\\Users\\DELL\\Kapil\\NLP\\NLP-Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\Kapil\\NLP\\NLP-Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [10]:
%pip install PyYAML

  Using cached PyYAML-6.0.1-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.1-cp312-cp312-win_amd64.whl (138 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install ensure

Note: you may need to restart the kernel to use updated packages.


In [8]:
from src.textsummarizer.constants import *
from src.textsummarizer.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )


        return data_ingestion_config

In [14]:
%pip install python-box
from box import Box




  Using cached python_box-7.1.1-py3-none-any.whl.metadata (7.8 kB)
Using cached python_box-7.1.1-py3-none-any.whl (28 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
import urllib.request as request
import zipfile
from src.textsummarizer.logging import logger
from src.textsummarizer.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-03 07:39:51,342: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-03 07:39:51,351: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-03 07:39:51,355: INFO: common: created directory at: artifacts]
[2024-02-03 07:39:51,358: INFO: common: created directory at: artifacts/data_ingestion]


[2024-02-03 07:40:19,205: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BBD8:18C353:789514:85D9C6:65BE5E46
Accept-Ranges: bytes
Date: Sat, 03 Feb 2024 15:39:52 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4721-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1706974791.462271,VS0,VE546
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 7d32626e73c639287a68c9e83309543b853b32ec
Expires: Sat, 03 Feb 2024 15:44:52 GMT
Source-Age: 0

]
